In [ ]:
import cv2 as cv
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
filenames = sorted(glob.glob("motion3_subset/*"))
print(len(filenames))
n = 840
step = len(filenames) // n
if step < 1:
    step = 1
filenames = filenames[0::step]
print(len(filenames))

In [ ]:
images = []

for image in filenames:
    images.append(cv.imread(image))


In [ ]:
# image grid
def image_grid(images, rows, cols):
    fig = plt.figure(figsize=(2 * cols, 2 * rows))
    for i in range(0, rows * cols):
        fig.add_subplot(rows, cols, i + 1)
        image = cv.cvtColor(images[i], cv.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.axis("off")
    plt.show()


In [ ]:
image_grid(images, 4, 10)

In [ ]:
CV_Stitcher_Status = {
    0: "OK",
    1: "ERR_NEED_MORE_IMGS",
    2: "ERR_HOMOGRAPHY_EST_FAIL",
    3: "ERR_CAMERA_PARAMS_ADJUST_FAIL",
}

In [ ]:
# homography matrix from extrinsic calibration
H = np.array([
    [
        -2.0034503550345025e-05, -0.00024135375115419462, -0.17586334553295266],
        [0.0009483766329084943, -3.283577271733423e-05, -0.3107424755962904],
        [-0.00016255750671592163, -0.006765916706019695, 1.0]
    ]
)

h, w = images[0].shape[:2]

T = np.array([
    [w, 0, -w/10],
    [0, h, h/2],
    [0, 0, 1]
])

TH = T @ H

In [ ]:
warped_images = []

for image in images:
    warped_image = cv.warpPerspective(image, TH, (w // 3, h))
    warped_images.append(warped_image)

In [ ]:
image_grid(warped_images, 4, 10)

In [ ]:
mode = cv.Stitcher_PANORAMA
mode = cv.Stitcher_SCANS

output_path = "output.png"

stitcher = cv.Stitcher.create(mode)
status, pano = stitcher.stitch(warped_images)

if status != cv.Stitcher_OK:
    print("Stitching failed with status", CV_Stitcher_Status[status])
else:
    cv.imwrite(output_path, pano)

In [ ]:
# also tried an incremental stitch

In [ ]:
def incremental_stitch(images, mode):
    stitcher = cv.Stitcher.create(mode)
    pano = None
    used = 0
    for image in images:
        if pano is None:
            pano = image
            continue
        try:
            status, pano_ = stitcher.stitch([pano, image])
        except cv.error:
            continue
        if status != cv.Stitcher_OK:
            continue
        else:
            used += 1
        pano = pano_
    return pano, used, len(images) - used

In [ ]:
pano, *_ = incremental_stitch(warped_images, cv.Stitcher_SCANS)
cv.imwrite("incremental_scan.png", pano)